In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone, ServerlessSpec

c:\Users\kevin\OneDrive\Documents\Learning\Youtube\GenAI\Chatbot_Project\GenAI_Charbot1\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()  # take env variable from .env

Pinecone_API_KEY_=os.getenv("Pinecone_API_KEY")
Pinecone_API_env_= os.getenv("Pinecone_API_env")
#print(key)
print(Pinecone_API_env_)

us-east-1


In [4]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

c:\Users\kevin\OneDrive\Documents\Learning\Youtube\GenAI\Chatbot_Project\GenAI_Charbot1\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\kevin\OneDrive\Documents\Learning\Youtube\GenAI\Chatbot_Project\GenAI_Charbot1\mchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# creating index
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=Pinecone_API_KEY_)
index_name = "med-chatbot"

index = pc.Index(index_name)
index.describe_index_stats()


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [7]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [8]:
 from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings,namespace="ns1")

retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

In [9]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [12]:
user_input='something'
while user_input!='quit':
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Allergy is an abnormal reaction to a substance, usually harmless, that occurs when the body's immune system overreacts to the presence of that substance. This overreaction can cause a variety of symptoms, including itchy eyes, nose, and throat, as well as sneezing, congestion, and coughing. In severe cases, an allergic reaction can lead to anaphylaxis, a life-threatening condition that requires immediate medical attention.
Response :  Inflammation is a process occurring in body tissues that is characterized by increased circulation and the accumulation of white blood cells. It can occur in disorders such as arthritis and cause harmful effects.
Response :  The ideal blood pressure is considered to be between 90-120/60-80 mmHg according to the American Heart Association.
Response :  It is important to understand that addiction is a chronic disease, and quitting can be challenging. However, there are many resources available to help individuals overcome their addiction. These 